### 📁 Available LoRa Models

| Model Size | Motion Type | LoRa Name | Description |
|-------|-------------|-----------|-------------|
| Lite, Pro | Object Rotation | `Microwave-right` | Rotates object ~360° to the right over 5 seconds |
| Lite, Pro | Object Rotation | `Microwave-left` | Rotates object ~360° to the left over 5 seconds |
| Lite, Pro | Camera Movement | `Arc-right` | Camera arcs ~360° around object to the right over 5 seconds |
| Lite, Pro | Camera Movement | `Arc-left` | Camera arcs ~360° around object to the left over 5 seconds |
| Lite | Camera Movement | `Dolly-in` | Camera moves forward toward subject |
| Lite | Camera Movement | `Dolly-out` | Camera moves backward from subject |
| Lite | Camera Movement | `Truck-right` | Camera tracks horizontally to the right |
| Lite | Camera Movement | `Truck-left` | Camera tracks horizontally to the left |

<br>

>  Note:
> 1. For optimal performance, select an appropriate LoRa for the **t2v** (text-to-video) or **i2v** (image-to-video) modality.  
>    We named our LoRas using the template: `kandinskylab/Kandinsky-5.0-{Modality}-{Model_Size}-LoRa-{LoRa_Name}` where `Modality` is either `I2V` or `T2V`.
> 2. Additional camera movements in the **Pro** model can be achieved directly via the prompt!

In [ ]:
import os
os.chdir("..")

import torch
from huggingface_hub import snapshot_download
from kandinsky import get_video_pipeline

from IPython.display import Video

### Load pipe 
first, ensure that you have downloaded the models using `download_models.py`

In [ ]:
pipe = get_video_pipeline(
    device_map={"dit": "cuda:0", "vae": "cuda:0", "text_embedder": "cuda:0"},
    conf_path="./configs/k5_lite_t2v_5s_sft_sd.yaml",
    mode='t2v',
)

#### Download LoRa adapter from HuggingFace🤗:

In [ ]:
cache_dir = './weights'
lora_name = 'Microwave-right'
repo_id = f"kandinskylab/Kandinsky-5.0-T2V-Lite-LoRa-{lora_name}"
adapter_path = snapshot_download(
    repo_id=repo_id,
    local_dir=os.path.join(cache_dir, repo_id),
    token=None
)

Load the adapter into the pipe:

In [ ]:
pipe.load_adapter(
    adapter_config=os.path.join(cache_dir, repo_id, "config_lora.json"),
    adapter_path= os.path.join(cache_dir, repo_id, "lora.safetensors"),
    adapter_name=lora_name
)

Note: The trigger words we use are stored in the LoRA safetensors metadata. They are automatically concatenated with prompts during inference.

In [ ]:
pipe.peft_triggers

Generate:

In [ ]:
save_dir = './outputs/loras/'
os.makedirs(save_dir, exist_ok=True)
out = pipe(
    "a cat in a red hat",
    time_length=5,
    width=768,
    height=512,
    save_path=os.path.join(save_dir, f't2v_lite_{lora_name}.mp4'),
    seed=1234,
)
Video(os.path.join(save_dir, f't2v_lite_{lora_name}.mp4'))

Download another adapter and switch to it:

In [ ]:
lora_name = 'Microwave-left'
repo_id = f"kandinskylab/Kandinsky-5.0-T2V-Lite-LoRa-{lora_name}"
adapter_path = snapshot_download(
    repo_id=repo_id,
    local_dir=os.path.join(cache_dir, repo_id),
    token=None
)
pipe.load_adapter(
    adapter_config=os.path.join(cache_dir, repo_id, "config_lora.json"),
    adapter_path= os.path.join(cache_dir, repo_id, "lora.safetensors"),
    adapter_name=lora_name
)

In [ ]:
out = pipe(
    "a cat in a red hat",
    time_length=5,
    width=768,
    height=512,
    save_path=os.path.join(save_dir, f't2v_lite_{lora_name}.mp4'),
    seed=1234,
)
Video(os.path.join(save_dir, f't2v_lite_{lora_name}.mp4'))

Switch to the previous LoRa adapter:

In [ ]:
prev_lora_name = 'Microwave-right'
pipe.set_adapter(prev_lora_name)

In [ ]:
out = pipe(
    "a cat in a red hat",
    time_length=5,
    width=768,
    height=512,
    save_path=os.path.join(save_dir, f't2v_lite_{prev_lora_name}_v2.mp4'),
    seed=1234,
)
Video(os.path.join(save_dir, f't2v_lite_{prev_lora_name}_v2.mp4'))

Disable all adapters:

In [ ]:
pipe.disable_adapters()

In [ ]:
out = pipe(
    "a cat in a red hat",
    time_length=5,
    width=768,
    height=512,
    save_path=os.path.join(save_dir, f'default.mp4'),
    seed=1234,
)
Video(os.path.join(save_dir, f'default.mp4'))